In [4]:
# first run this setup file
%run -i C:/Users/champ/Python_proj/base_financial_repo/import_statements # import statements
%run -i C:/Users/champ/Documents/Pers/Python/stock_dfs/personal_vars_and_constants # initial global vars and constants file    
%run -i C:/Users/champ/Python_proj/base_financial_repo/global_vars_and_constants # initial global vars and constants file
%run -i C:/Users/champ/Python_proj/base_financial_repo/functions # initial function file

In [6]:
# Relative strength indicator and economic indicators 
%matplotlib widget

# Local variables
accounts = ['sectors','inx']
Inx = 'spy'
data_type='Adj Close'
force_update = False # Force download of financial data

for account in accounts: 
    prices_file = MAIN_FOLDER+META_FILE_FOLDER+'prices_'+account+'.csv'
    if account == 'sectors': 
        securities = ['xlp', 'xle', 'xlv', 'xli', 'xlb',
                      'xlk', 'iyt', 'xlu', 'vcr', 'vfh', 'vnq']
        sector_price_data = get_prices(securities, account, prices_file, 
                                       data_type, start_date, force_update)
        
    if account == 'inx':
        securities = [Inx]
        inx_price_data = get_prices(securities,
                                    account, prices_file,
                                    data_type, start_date,
                                    force_update)
        inx_price_data['MA50_index']=inx_price_data[Inx].rolling(50).mean()
        inx_price_data['MA200_index']=inx_price_data[Inx].rolling(200).mean()
        inx_price_data.index = pd.to_datetime(inx_price_data.index)

#  Function calls
Market_indicator_signal,mkt_ind = get_mkt_indicator(inx_price_data)
Average_return = _1_3_6_9_12_month_returns(sector_price_data)
FRED_data_mort = get_FRED_data('MORTGAGE30US','mort',start_date,0,1)
FRED_data_LEI = get_FRED_data('USSLIND','LEI',start_date,0,1)
FRED_data_inf = get_FRED_data('CPILFESL','inf',start_date,12,100)
FRED_data_10_2 = get_FRED_data('T10Y2Y','10_2',start_date,0,1)
FRED_data_RF = get_FRED_data('DTB3','RF',start_date,0,1)
FRED_data_ICSA = get_FRED_data('ICSA','UnEmp',start_date,52,1)

combined_12_mo_return = pd.concat([FRED_data_RF, FRED_data_inf,
                                  FRED_data_LEI, FRED_data_mort,
                                  FRED_data_10_2, FRED_data_ICSA], 
                                  axis=1, sort=False)
combined_12_mo_return = combined_12_mo_return.fillna(method='ffill')
combined_12_mo_return = combined_12_mo_return.fillna(method='bfill')

#  Display
display = display_relative_strength(Average_return,
                                    Market_indicator_signal,
                                    mkt_ind,
                                    Securities_file_location)
display2 = prepare_market_indicator(inx_price_data,Inx);
display3 = bollinger_bands_graph(inx_price_data,Inx);
display4 = econometric_graph(combined_12_mo_return);

Today the market was open and the 
sectors price data is updated

Last run: 2020-06-24 15:56:38.702349
The last 17 years of price data
Market analysis end timestamp : 15

The ascending mean return for 1,3,6,12mo ranking for 11 sectors are :
Ticker  Av_ret            category  Div yeild / yr  Div - exp  Expense ratio   EX_date_x Future_EX_date  Days_left     reg_Vol
   xlk    23.2          Technology            1.22       1.09           0.13  2020-06-22     2020-10-11      109.0  12537361.0
   vcr    13.9   Consumer Cyclical            1.14       1.04           0.10  2020-06-22     2020-10-24      122.0    121137.0
   xlv     6.7              Health            2.18       2.05           0.13  2020-06-22     2020-10-11      109.0  15670699.0
   xlb     1.8   Natural Resources            2.24       2.11           0.13  2020-06-22     2020-10-11      109.0   5652940.0
   xlp     1.5  Consumer Defensive            2.71       2.58           0.13  2020-06-22     2020-10-11      109.0  13971456

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …